In [6]:
import requests
import pandas as pd

# Definerer klient-id, personlig for Georg Rommetveit
client_id = "5b9e3b06-3d3d-4049-9b86-b52c0e8cfb81"

# Definerer endepunkt og parametre for forespørsel
endpoint = 'https://frost.met.no/observations/v0.jsonld'
parameters = {
    'sources': 'SN18700,SN90450',
    'elements': 'sum(precipitation_amount P1D), sum(duration_of_precipitation P1D), sum(duration_of_sunshine P1D), sea_water_speed, mean(air_temerature P1D), mean(soil_temperature P1D)', # Nedbør, sol, vannhastighet og temperatur  
    'referencetime': '2015-01-01/2025-01-01',
}
# Opretter en forespørsel til frost.met.no
r = requests.get(endpoint, parameters, auth=(client_id,''))

# Henter json data
json = r.json()

# Bekrefter om data er hentet
if r.status_code == 200:
    data = json['data']
    print('Data retrieved from frost.met.no!')
else:
    print('Error! Returned status code %s' % r.status_code)
    print('Message: %s' % json['error']['message'])
    print('Reason: %s' % json['error']['reason'])



Data retrieved from frost.met.no!


In [7]:
import os
import pandas as pd

output_filename = "met_data.csv"

# Konverter JSON-data til en pandas DataFrame
try:
    data_expanded = []
    for entry in data:
        source_id = entry.get("sourceId", "N/A")  # Henter kilde-ID
        ref_time = entry.get("referenceTime", "N/A")  # Henter tid
        
        for obs in entry.get("observations", []):
            obs["sourceId"] = source_id  # Legger til sourceId
            obs["referenceTime"] = ref_time  # Legger til tidsstempel
            data_expanded.append(obs)
    df = pd.json_normalize(data_expanded)
except Exception as e:
    print(f"Feil ved konvertering av JSON til DataFrame: {e}")

# Håndter manglende data ved å fylle med "N/A"
df.fillna("N/A", inplace=True)


# Lagre DataFrame som CSV
output_path = os.path.join(os.getcwd(), "..", "..", "data", output_filename)
try:
    df.to_csv(output_path, index=False, encoding="utf-8", header=False)
    print(f"Data lagret som CSV i {output_path}")
except Exception as e:
    print(f"Feil ved lagring av CSV-fil: {e}") # Hever feil dersom lagring feiler


Data lagret som CSV i /Users/georgrommetveit/Documents/anvendtprog/miljodataAnalyse/src/API/../../data/met_data.csv


/var/folders/z7/44tqr8zx7vd2w_1l30pcncwr0000gn/T/ipykernel_43004/918675783.py:22: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'N/A' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.fillna("N/A", inplace=True)
